In [ ]:
!apt-get -y update
!apt-get -y install mpich
!pip install mpi4py

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
mpich is already the newest version (3.3.2-2build1).
0 upgraded, 0 newly installed, 0 to remove and 27 

###Task-1 : Calculate rank and number of processor

In [ ]:
%%writefile mpi_r.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
  int rank, size;

  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);

  printf("I am process %d of %d.\n", rank, size);

  MPI_Finalize();
  return 0;
}


Overwriting mpi_r.c


In [ ]:

!mpicc mpi_r.c -o mpi_r
!mpirun --allow-run-as-root -n 1 ./mpi_r

I am process 0 of 1.


###Task-2 : Calculate value of pi 

In [ ]:
%%writefile mpi_pi.c
#include <mpi.h>
#include <stdio.h>
#include <math.h>

int main(int argc, char** argv) {
  int rank, size;
  long long int n = 1000000;
  double x, pi, sum = 0.0;
  int i;

  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);

  for (i = rank; i < n; i += size) {
    x = (double)(i + 0.5) / (double)n;
    sum += 4.0 / (1.0 + x * x);
  }

  double local_pi = sum / n;
  MPI_Reduce(&local_pi, &pi, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);

  if (rank == 0) {
    printf("pi is approximately %.16f.\n", pi);
  }

  MPI_Finalize();
  return 0;
}


Overwriting mpi_pi.c


In [ ]:
!mpicc mpi_pi.c -o mpi_pi
!mpirun --allow-run-as-root -np 1 ./mpi_pi


pi is approximately 3.1415926535897643.


###Advanced MPI program that has a total number of 4 processes, where the process with
###rank = 0 should send VJTI letter to all the processes using MPI_Scatter call.

In [ ]:
%%writefile mpi_vjti.c
#include <mpi.h>
#include <stdio.h>
#include <string.h>

#define MESSAGE_SIZE 5

int main(int argc, char** argv) {
  int rank, size;
  char message[MESSAGE_SIZE + 1];

  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);

  if (rank == 0) {
    strcpy(message, "VJTI");
  }

  MPI_Scatter(message, MESSAGE_SIZE, MPI_CHAR, message, MESSAGE_SIZE, MPI_CHAR, 0, MPI_COMM_WORLD);

  printf("Process %d received message: %s\n", rank, message);

  MPI_Finalize();
  return 0;
}


Overwriting mpi_vjti.c


In [ ]:
!mpicc mpi_vjti.c -o mpi_vjti
!mpirun --allow-run-as-root -np 1 ./mpi_vjti


Process 0 received message: VJTI


###Task-4: Calculate maximum of array

In [ ]:
%%writefile mpi_maximum.c
#include <mpi.h>
#include <stdio.h>

#define ARRAY_SIZE 6

int main(int argc, char** argv) {
  int rank, size;
  int array[ARRAY_SIZE] = {1, 6, 3, 8, 2, 7};
  
  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);

  int local_max = array[rank];
  int max;

  MPI_Reduce(&local_max, &max, 1, MPI_INT, MPI_MAX, 0, MPI_COMM_WORLD);

  if (rank == 0) {
    printf("The maximum value in the array is: %d\n", max);
  }

  MPI_Finalize();
  return 0;
}


Overwriting mpi_maximum.c


In [ ]:
!mpicc mpi_maximumm.c -o mpi_maximum
!mpirun --allow-run-as-root -np 1 ./mpi_maximum

--------------------------------------------------------------------------
Primary job  terminated normally, but 1 process returned
a non-zero exit code. Per user-direction, the job has been aborted.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
mpirun noticed that process rank 0 with PID 0 on node 3342d2aed770 exited on signal 11 (Segmentation fault).
--------------------------------------------------------------------------


###Task-5: Ring Topology

In [ ]:
%%writefile mpi_ring.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
  int rank, size;
  int message_in, message_out;

  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);

  if (rank == 0) {
    message_out = 1;
    MPI_Send(&message_out, 1, MPI_INT, (rank + 1) % size, 0, MPI_COMM_WORLD);
    MPI_Recv(&message_in, 1, MPI_INT, size - 1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    printf("Process %d received message %d from process %d.\n", rank, message_in, size - 1);
  } else {
    MPI_Recv(&message_in, 1, MPI_INT, rank - 1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    printf("Process %d received message %d from process %d.\n", rank, message_in, rank - 1);
    if (rank == size - 1) {
      message_out = 0;
    } else {
      message_out = message_in + 1;
    }
    MPI_Send(&message_out, 1, MPI_INT, (rank + 1) % size, 0, MPI_COMM_WORLD);
  }

  MPI_Finalize();
  return 0;
}


Overwriting mpi_ring.c


In [ ]:
!mpicc mpi_ring.c -o mpi_ring
!mpirun --allow-run-as-root -np 1 ./mpi_ring

Process 0 received message 1 from process 0.
